In [93]:
from langchain.document_loaders import PyPDFLoader
loader= PyPDFLoader("acsbr-015.pdf")
docs = loader.load()
# docs

In [94]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter  = RecursiveCharacterTextSplitter(chunk_size=700,chunk_overlap=100)
final_docs = text_splitter.split_documents(docs)

In [95]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["GEMINI_API_KEY"]=os.getenv("GEMINI_API_KEY")
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=os.getenv("GEMINI_API_KEY"))

In [96]:
from langchain_community.vectorstores import FAISS
vector_data = FAISS.from_documents(final_docs,embeddings)
retriver = vector_data.as_retriever(search_kwargs={"k":10})
retriver

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001FBC9E57130>, search_kwargs={'k': 10})

In [97]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(""""
                        Answer the Question Based only on Provided Context:
                        Do not use Based on the provided context like sentence.
                        Try to give answer in deatil.
                        Give Accurate Answer and use only detail which is mentioned  in the context.
                        <context>
                        {context}
                        </context>
                        Question : {question}
""")

In [98]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",google_api_key=os.getenv("GEMINI_API_KEY"))

In [99]:
from langchain.chains import RetrievalQA

retrivalQA = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriver,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt},
)
print(retrivalQA.input_keys)


['query']


In [101]:
response = retrivalQA.invoke({"query":"In 2022, how amny states saw increases in public health insurance coverage?"})
response['result']

'In 2022, thirteen states saw increases in public health insurance coverage. Increases in public health insurance coverage ranged from 0.6 percentage points to 2.3 percentage points across states.'